In [ ]:
import aiohttp
from connector.bybit.bybit_perp import BybitPerp

connector = aiohttp.TCPConnector(keepalive_timeout=30, enable_cleanup_closed=True, limit_per_host=10)  # TTL idle-сокета  # подчистка half-open SSL
timeout = aiohttp.ClientTimeout(total=10)  # общий тайм-аут запроса
session = aiohttp.ClientSession(connector=connector, timeout=timeout)
bp = BybitPerp(session=session)
await bp.start_tasks()


In [ ]:
import aiohttp
from connector.binance.binance_perp import BinancePerp

connector = aiohttp.TCPConnector(keepalive_timeout=30, enable_cleanup_closed=True, limit_per_host=10)  # TTL idle-сокета  # подчистка half-open SSL
timeout = aiohttp.ClientTimeout(total=10)  # общий тайм-аут запроса
session = aiohttp.ClientSession(connector=connector, timeout=timeout)
bp = BinancePerp(session=session)
await bp.start_tasks()

In [ ]:
print(len(bp.instruments))
bp.instruments

In [ ]:
BASE_URL = "https://api.bybit.com"
EXCHANGE_INFO_URL = "/v5/market/instruments-info"
import aiohttp
from connector.connector import HTTPMethod

connector = aiohttp.TCPConnector(keepalive_timeout=30, enable_cleanup_closed=True, limit_per_host=10)  # TTL idle-сокета  # подчистка half-open SSL
timeout = aiohttp.ClientTimeout(total=10)  # общий тайм-аут запроса
session = aiohttp.ClientSession(connector=connector, timeout=timeout)
params={"category": "linear"}

async with session.request(method=HTTPMethod.GET.value, url=BASE_URL+EXCHANGE_INFO_URL, params=params) as response:
    r = await response.json()
await session.close()

In [ ]:
for row in r["result"]["list"]:
    if row["contractType"] == "LinearPerpetual" and row["status"] == "Trading" and row["quoteCoin"] == "USDT":
        print(row["symbol"])

In [1]:
import aiohttp
from connector.connector import HTTPMethod
import time
import asyncio

BASE_URL = "https://fapi.binance.com/fapi/"
EXCHANGE_INFO_URL = "v1/exchangeInfo"
CONSTITUENS_URL = "v1/constituents"
url_server_time = "v1/time"

connector = aiohttp.TCPConnector(keepalive_timeout=30, enable_cleanup_closed=True, limit_per_host=10)  # TTL idle-сокета  # подчистка half-open SSL
timeout = aiohttp.ClientTimeout(total=10)  # общий тайм-аут запроса
session = aiohttp.ClientSession(connector=connector, timeout=timeout)

async with session.request(method=HTTPMethod.GET.value, url=BASE_URL+url_server_time, params={}) as response:
    r = await response.json()
delay = time.time() - r["serverTime"] / 1000
print(delay)

async with session.request(method=HTTPMethod.GET.value, url=BASE_URL+EXCHANGE_INFO_URL, params={}) as response:
    r = await response.json()

exchanges: dict[str, int] = {}
exchange_weights: dict[str, float] = {}
symbols: list[str] = []
error_symbols: list[str] = []
for inst_info in r["symbols"]:
    if inst_info["status"] == "TRADING" and inst_info["contractType"] == "PERPETUAL" and inst_info["quoteAsset"] == "USDT":
        symbols.append(inst_info["symbol"])

for i, symbol in enumerate(symbols):
    params = {"symbol": symbol}
    dt = time.time()
    async with session.request(method=HTTPMethod.GET.value, url=BASE_URL+CONSTITUENS_URL, params=params) as response:
        r = await response.json()
        
    try:
        for info in r["constituents"]:
            exchange = info["exchange"]
            exchanges[exchange] = exchanges.get(exchange, 0) + 1
            exchange_weights[exchange] = exchange_weights.get(exchange, 0.0) + float(info["weight"])
    except:
        error_symbols.append(symbol)


    used_limit = int(response.headers["x-mbx-used-weight-1m"])
    if used_limit > 2300:
        secs_to_free = max(0, dt//60+60 - delay - dt + 1)
        if secs_to_free:
            await asyncio.sleep(secs_to_free)

    print(f"used_weight: {used_limit}, symbol_id: {i}", " "*30, end="\r")

await session.close()

0.1896228790283203


In [4]:
rows = [[exch, w] for exch, w in sorted(exchange_weights.items(), key=lambda kv: kv[1], reverse=True)]
headers = ("Exchange", "Weight")
col1 = max(len(headers[0]), *(len(k) for k, _ in rows))
col2 = max(len(headers[1]), *(len(f"{v:.6g}") for _, v in rows))

line = f"+-{'-'*col1}-+-{'-'*col2}-+"
print(line)
print(f"| {headers[0]:<{col1}} | {headers[1]:>{col2}} |")
print(line)
for k, v in rows:
    print(f"| {k:<{col1}} | {v:>{col2}.6g} |")
print(line)

+----------------+-----------+
| Exchange       |    Weight |
+----------------+-----------+
| binance        |   227.735 |
| mxc            |   50.6816 |
| bitget         |   44.1354 |
| gateio         |   39.6993 |
| okex           |   34.4464 |
| kucoin         |    34.086 |
| bybit          |   29.5379 |
| binance_alpha  |   21.1888 |
| pancakeswapV3  |   12.7179 |
| coinbase       |    10.574 |
| binance_future |   10.4737 |
| binance_cross2 |   6.34885 |
| binance_cross  |   5.44228 |
| aster          |   1.32798 |
| bitmax         |  0.838995 |
| pancakeswapV2  |  0.454386 |
| binance_cross3 |  0.403323 |
| bitfinex       |       0.4 |
| uniswapV3      |      0.35 |
| hitbtc         | 0.0827508 |
| uniswapV2      |  0.075641 |
+----------------+-----------+


In [7]:
rows = [[exch, w] for exch, w in sorted(exchanges.items(), key=lambda kv: kv[1], reverse=True)]
headers = ("Exchange", "Count")
col1 = max(len(headers[0]), *(len(k) for k, _ in rows))
col2 = max(len(headers[1]), *(len(f"{v:.6g}") for _, v in rows))

line = f"+-{'-'*col1}-+-{'-'*col2}-+"
print(line)
print(f"| {headers[0]:<{col1}} | {headers[1]:>{col2}} |")
print(line)
for k, v in rows:
    print(f"| {k:<{col1}} | {v:>{col2}.6g} |")
print(line)

+----------------+-------+
| Exchange       | Count |
+----------------+-------+
| mxc            |   414 |
| gateio         |   411 |
| binance        |   376 |
| bitget         |   359 |
| kucoin         |   292 |
| bybit          |   243 |
| okex           |   197 |
| binance_cross2 |   124 |
| binance_future |    66 |
| coinbase       |    55 |
| pancakeswapV3  |    53 |
| binance_alpha  |    51 |
| binance_cross  |    45 |
| binance_cross3 |    14 |
| bitmax         |     8 |
| aster          |     5 |
| hitbtc         |     2 |
| pancakeswapV2  |     2 |
| uniswapV2      |     2 |
| uniswapV3      |     2 |
| bitfinex       |     1 |
+----------------+-------+


In [ ]:
inst_info["baseAsset"]

'币安人生'